In [ ]:
import os
os.makedirs('templates', exist_ok=True)
os.makedirs('static', exist_ok=True)

html_content = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Loan Eligibility Prediction</title>
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    .container {
      margin-top: 30px;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1 class="text-center">Loan Eligibility Prediction Project</h1>
    <hr>

    <!-- File Upload & Model Training Form -->
    <div class="card mb-4">
      <div class="card-header">
        Step 1: Upload Dataset & Train Model
      </div>
      <div class="card-body">
        <form id="uploadForm" enctype="multipart/form-data">
          <div class="form-group">
            <label for="file">Upload your CSV file (e.g., loan-test.csv):</label>
            <input type="file" class="form-control-file" id="file" name="file" accept=".csv">
          </div>
          <button type="submit" class="btn btn-primary">Upload & Train</button>
        </form>
        <div id="uploadResult" class="mt-3"></div>
      </div>
    </div>

    <!-- Display Confusion Matrix -->
    <div class="card mb-4">
      <div class="card-header">
        Model Evaluation - Confusion Matrix
      </div>
      <div class="card-body">
        <img id="confusionMatrix" src="" alt="Confusion Matrix" style="width:100%; max-width:600px; display:none;">
      </div>
    </div>

    <!-- Loan Prediction Form -->
    <div class="card">
      <div class="card-header">
        Step 2: Predict Loan Eligibility
      </div>
      <div class="card-body">
        <form id="predictForm">
          <div class="form-row">
            <!-- Using dropdowns for binary values -->
            <div class="form-group col-md-4">
              <label for="Gender">Gender</label>
              <select class="form-control" id="Gender" name="Gender" required>
                <option value="0">0 (Female)</option>
                <option value="1">1 (Male)</option>
              </select>
            </div>
            <div class="form-group col-md-4">
              <label for="Married">Married</label>
              <select class="form-control" id="Married" name="Married" required>
                <option value="0">0 (No)</option>
                <option value="1">1 (Yes)</option>
              </select>
            </div>
            <div class="form-group col-md-4">
              <label for="Dependents">Dependents</label>
              <input type="number" class="form-control" id="Dependents" name="Dependents" required>
            </div>
          </div>
          <div class="form-row">
            <div class="form-group col-md-4">
              <label for="Education">Education</label>
              <select class="form-control" id="Education" name="Education" required>
                <option value="0">0 (Not Graduate)</option>
                <option value="1">1 (Graduate)</option>
              </select>
            </div>
            <div class="form-group col-md-4">
              <label for="Self_Employed">Self Employed</label>
              <select class="form-control" id="Self_Employed" name="Self_Employed" required>
                <option value="0">0 (No)</option>
                <option value="1">1 (Yes)</option>
              </select>
            </div>
            <div class="form-group col-md-4">
              <label for="Property_Area">Property Area</label>
              <select class="form-control" id="Property_Area" name="Property_Area" required>
                <option value="0">0</option>
                <option value="1">1</option>
                <option value="2">2</option>
              </select>
            </div>
          </div>
          <div class="form-row">
            <div class="form-group col-md-4">
              <label for="ApplicantIncome">Applicant Income</label>
              <input type="number" class="form-control" id="ApplicantIncome" name="ApplicantIncome" required>
            </div>
            <div class="form-group col-md-4">
              <label for="CoapplicantIncome">Coapplicant Income</label>
              <input type="number" class="form-control" id="CoapplicantIncome" name="CoapplicantIncome" required>
            </div>
            <div class="form-group col-md-4">
              <label for="LoanAmount">Loan Amount</label>
              <input type="number" class="form-control" id="LoanAmount" name="LoanAmount" required>
            </div>
          </div>
          <div class="form-row">
            <div class="form-group col-md-6">
              <label for="Loan_Amount_Term">Loan Amount Term</label>
              <input type="number" class="form-control" id="Loan_Amount_Term" name="Loan_Amount_Term" required>
            </div>
            <div class="form-group col-md-6">
              <label for="Credit_History">Credit History</label>
              <select class="form-control" id="Credit_History" name="Credit_History" required>
                <option value="0">0</option>
                <option value="1">1</option>
              </select>
            </div>
          </div>
          <button type="submit" class="btn btn-success">Predict Loan Eligibility</button>
        </form>
        <div id="predictionResult" class="mt-3"></div>
      </div>
    </div>
  </div>

  <!-- jQuery and AJAX handling -->
  <script src="https://code.jquery.com/jquery-3.4.1.min.js"></script>
  <script>
    $(document).ready(function(){
      // Handle dataset upload and training
      $("#uploadForm").on("submit", function(e){
        e.preventDefault();
        var formData = new FormData(this);
        $.ajax({
          url: "/upload",
          type: "POST",
          data: formData,
          contentType: false,
          processData: false,
          success: function(response) {
            $("#uploadResult").html("<div class='alert alert-success'>" + response.message +
              "<br>Accuracy: " + (response.accuracy * 100).toFixed(2) + "%</div>");
            $("#confusionMatrix").attr("src", "/static/confusion_matrix.png").show();
          },
          error: function(xhr) {
            console.log(xhr);
            $("#uploadResult").html("<div class='alert alert-danger'>Error: " + xhr.responseText + "</div>");
          }
        });
      });

      // Handle prediction submission
      $("#predictForm").on("submit", function(e){
        e.preventDefault();
        var formData = {
          Gender: Number($("#Gender").val()),
          Married: Number($("#Married").val()),
          Dependents: Number($("#Dependents").val()),
          Education: Number($("#Education").val()),
          Self_Employed: Number($("#Self_Employed").val()),
          Property_Area: Number($("#Property_Area").val()),
          ApplicantIncome: Number($("#ApplicantIncome").val()),
          CoapplicantIncome: Number($("#CoapplicantIncome").val()),
          LoanAmount: Number($("#LoanAmount").val()),
          Loan_Amount_Term: Number($("#Loan_Amount_Term").val()),
          Credit_History: Number($("#Credit_History").val())
        };
        $.ajax({
          url: "/predict",
          type: "POST",
          contentType: "application/json",
          data: JSON.stringify(formData),
          success: function(response) {
            $("#predictionResult").html("<div class='alert alert-info'>" + response.result + "</div>");
          },
          error: function(xhr) {
            console.log(xhr);
            $("#predictionResult").html("<div class='alert alert-danger'>Error: " + xhr.responseText + "</div>");
          }
        });
      });
    });
  </script>
</body>
</html>
"""

with open('templates/index.html', 'w') as f:
    f.write(html_content)


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib, os, io
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

model = None
data_df = None

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    global data_df, model
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    try:
        # Read CSV and fill missing values
        data_df = pd.read_csv(io.StringIO(file.stream.read().decode("UTF8")), delimiter=',')
        data_df.fillna({
            'Gender': data_df['Gender'].mode()[0],
            'Married': data_df['Married'].mode()[0],
            'Dependents': data_df['Dependents'].mode()[0],
            'Self_Employed': data_df['Self_Employed'].mode()[0],
            'LoanAmount': data_df['LoanAmount'].mean(),
            'Loan_Amount_Term': data_df['Loan_Amount_Term'].mode()[0],
            'Credit_History': data_df['Credit_History'].mode()[0],
        }, inplace=True)

        # Create Target Column 'Loan_Status'
        data_df['Loan_Status'] = data_df.apply(
            lambda row: 1 if row['Credit_History'] == 1 and row['LoanAmount'] <= 200 else 0, axis=1
        )

        # Encode categorical columns
        label_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents']
        for col in label_cols:
            data_df[col] = LabelEncoder().fit_transform(data_df[col])

        # Split into features and target; adjust column names if needed
        if 'Loan_ID' in data_df.columns:
            X = data_df.drop(columns=['Loan_ID', 'Loan_Status'])
        else:
            X = data_df.drop(columns=['Loan_Status'])
        y = data_df['Loan_Status']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train model
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Create confusion matrix plot
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.savefig(os.path.join("static", "confusion_matrix.png"))
        plt.close()

        # Save model to a file
        joblib.dump(model, 'loan_eligibility_model.pkl')
        return jsonify({'message': 'Model trained successfully!', 'accuracy': acc})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/predict', methods=['POST'])
def predict():
    global model
    if model is None:
        if os.path.exists('loan_eligibility_model.pkl'):
            model = joblib.load('loan_eligibility_model.pkl')
        else:
            return jsonify({'error': 'Model not found. Please upload dataset and train model first.'}), 400
    try:
        user_input = request.json
        input_df = pd.DataFrame([user_input])
        result = model.predict(input_df)[0]
        prediction_text = "✅ Loan is Eligible" if result == 1 else "❌ Loan is Not Eligible"
        return jsonify({'result': prediction_text})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run()


Writing app.py


In [ ]:
!python app.py


Traceback (most recent call last):
  File "/content/app.py", line 10, in <module>
    from flask_ngrok import run_with_ngrok
ModuleNotFoundError: No module named 'flask_ngrok'


In [ ]:
!head -n 30 templates/index.html


<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Loan Eligibility Prediction</title>
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    .container {
      margin-top: 30px;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1 class="text-center">Loan Eligibility Prediction Project</h1>
    <hr>

    <!-- File Upload & Model Training Form -->
    <div class="card mb-4">
      <div class="card-header">
        Step 1: Upload Dataset & Train Model
      </div>
      <div class="card-body">
        <form id="uploadForm" enctype="multipart/form-data">
          <div class="form-group">
            <label for="file">Upload your CSV file (e.g., loan-test.csv):</label>
            <input type="file" class="form-control-file" id="file" name="file" accept=".csv">
          </div>
          <button type="submit" class="btn btn-primary">Upload & Train</button>
        </form>


In [ ]:
from IPython.display import HTML  # Import the HTML class
display(HTML(html_content))